In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date

In [2]:
spark = SparkSession.builder \
    .appName("App") \
    .master("spark://master:7077") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.executor.cores", "3") \
    .config("spark.executor.instances", "4") \
    .config("spark.sql.shuffle.partitions", "100") \
    .getOrCreate()

spark.conf.set("spark.sql.files.ignoreCorruptFiles", "true")
spark.conf.set("spark.sql.files.ignoreMissingFiles", "true")# total across executors

In [3]:
import requests
from datetime import datetime
SPARK_UI = "http://192.168.56.102:4040"

## Von hdfs lesen

10 Wiederholungen

In [5]:
import math
import time
import numpy as np
listh10 = []

start_all = time.time()
for i in range(10):
    dfrh= spark.read.parquet("hdfs:///input/parquet")
    app_id = requests.get(f"{SPARK_UI}/api/v1/applications").json()[0]["id"]
    job = requests.get(f"{SPARK_UI}/api/v1/applications/{app_id}/jobs").json()[0]
    start = datetime.fromisoformat(job["submissionTime"].replace("GMT", "+00:00"))
    end = datetime.fromisoformat(job["completionTime"].replace("GMT", "+00:00"))
    timer = (end - start).total_seconds()
    job_info = requests.get(f"{SPARK_UI}/api/v1/applications/{app_id}/jobs").json()
    latest_job = sorted(job_info, key=lambda j: j["submissionTime"], reverse=True)[0]
    listh10.append(timer)
    print(f"Job ID: {latest_job['jobId']} braucht {timer:.2f} Sekunden")
end_all = time.time() 

print(listh10)
print(f"Zeit für die Ausführung der Zelle: {end_all - start_all:.2f} seconds")
print(f"Mittelwert aller Spark Jobs: {np.mean(listh10):.2f} seconds")
print(f"Median aller Spark Jobs: {np.median(listh10):.2f} seconds")
print(f"Sdt aller Spark Jobs: {np.std(listh10):.2f} seconds")

Job ID: 10 braucht 0.12 Sekunden
Job ID: 11 braucht 0.14 Sekunden
Job ID: 12 braucht 0.14 Sekunden
Job ID: 13 braucht 0.16 Sekunden
Job ID: 14 braucht 0.14 Sekunden
Job ID: 15 braucht 0.12 Sekunden
Job ID: 16 braucht 0.11 Sekunden
Job ID: 17 braucht 0.07 Sekunden
Job ID: 18 braucht 0.08 Sekunden
Job ID: 19 braucht 0.12 Sekunden
[0.122, 0.138, 0.14, 0.163, 0.143, 0.115, 0.107, 0.07, 0.084, 0.125]
Zeit für die Ausführung der Zelle: 2.90 seconds
Mittelwert aller Spark Jobs: 0.12 seconds
Median aller Spark Jobs: 0.12 seconds
Sdt aller Spark Jobs: 0.03 seconds


100 Wiederholungen

In [6]:
import math
import time
import numpy as np
listh100 = []

start_all = time.time()
for i in range(100):
    dfrh= spark.read.parquet("hdfs:///input/parquet")
    app_id = requests.get(f"{SPARK_UI}/api/v1/applications").json()[0]["id"]
    job = requests.get(f"{SPARK_UI}/api/v1/applications/{app_id}/jobs").json()[0]
    start = datetime.fromisoformat(job["submissionTime"].replace("GMT", "+00:00"))
    end = datetime.fromisoformat(job["completionTime"].replace("GMT", "+00:00"))
    timer = (end - start).total_seconds()
    job_info = requests.get(f"{SPARK_UI}/api/v1/applications/{app_id}/jobs").json()
    latest_job = sorted(job_info, key=lambda j: j["submissionTime"], reverse=True)[0]
    listh100.append(timer)
    print(f"Job ID: {latest_job['jobId']} took {timer:.2f} seconds")
end_all = time.time()

print(listh100)
print(f"Zeit für die Ausführung der Zelle: {end_all - start_all:.2f} seconds")
print(f"Mittelwert aller Spark Jobs: {np.mean(listh100):.2f} seconds")
print(f"Median aller Spark Jobs: {np.median(listh100):.2f} seconds")
print(f"Sdt aller Spark Jobs: {np.std(listh100):.2f} seconds")

Job ID: 20 took 0.11 seconds
Job ID: 21 took 0.11 seconds
Job ID: 22 took 0.15 seconds
Job ID: 23 took 0.10 seconds
Job ID: 24 took 0.17 seconds
Job ID: 25 took 0.13 seconds
Job ID: 26 took 0.13 seconds
Job ID: 27 took 0.09 seconds
Job ID: 28 took 0.10 seconds
Job ID: 29 took 0.09 seconds
Job ID: 30 took 0.11 seconds
Job ID: 31 took 0.09 seconds
Job ID: 32 took 0.07 seconds
Job ID: 33 took 0.14 seconds
Job ID: 34 took 0.15 seconds
Job ID: 35 took 0.14 seconds
Job ID: 36 took 0.14 seconds
Job ID: 37 took 0.15 seconds
Job ID: 38 took 0.18 seconds
Job ID: 39 took 0.12 seconds
Job ID: 40 took 0.28 seconds
Job ID: 41 took 0.09 seconds
Job ID: 42 took 0.11 seconds
Job ID: 43 took 0.08 seconds
Job ID: 44 took 0.09 seconds
Job ID: 45 took 0.14 seconds
Job ID: 46 took 0.13 seconds
Job ID: 47 took 0.14 seconds
Job ID: 48 took 0.12 seconds
Job ID: 49 took 0.16 seconds
Job ID: 50 took 0.13 seconds
Job ID: 51 took 0.13 seconds
Job ID: 52 took 0.14 seconds
Job ID: 53 took 0.12 seconds
Job ID: 54 too

## Count testen

In [6]:
dfrh= spark.read.parquet("hdfs:///input/parquet")

In [7]:
dfrh.createOrReplaceTempView("my_table")

Einfacher select und count, ohne Ausgabe

In [10]:
import math
import time
import requests
from datetime import datetime

listcount = []

start_all = time.time()
def get_all_jobs():
    app_id = requests.get(f"{SPARK_UI}/api/v1/applications").json()[0]["id"]
    job_info = requests.get(f"{SPARK_UI}/api/v1/applications/{app_id}/jobs").json()
    return job_info

for i in range(10):
    # Step 1: Snapshot of current jobs
    jobs_before = {j["jobId"] for j in get_all_jobs()}
    
    query = f"""
    SELECT gbifID, COUNT(gbifID) AS count
    FROM my_table
    WHERE gbifID IS NOT NULL
    GROUP BY gbifID
    ORDER BY count DESC
"""

# Create Spark DataFrame
    spark_df = spark.sql(query)

# Show the result in Spark (default top 20 rows)
    spark_df.count()
    
    # Step 3: Wait to ensure Spark UI is updated
    time.sleep(2)
    
    # Step 4: Get new jobs
    jobs_after = get_all_jobs()
    new_jobs = [j for j in jobs_after if j["jobId"] not in jobs_before]
    
    # Step 5: Sum duration of all new jobs
    total_duration = 0.0
    for job in new_jobs:
        start = datetime.fromisoformat(job["submissionTime"].replace("GMT", "+00:00"))
        end = datetime.fromisoformat(job["completionTime"].replace("GMT", "+00:00"))
        duration = (end - start).total_seconds()
        total_duration += duration
        print(f"  Included Job {job['jobId']} took {duration:.2f}s")

    listcount.append(total_duration)
end_all = time.time()
# Final average
print("Average time over 10 queries:", round(math.fsum(listcount)/len(listcount), 2), "seconds")
print(listcount)
print(f"Zeit für die Ausführung der Zelle: {end_all - start_all:.2f} seconds")
print(f"Mittelwert aller Spark Jobs: {np.mean(listcount):.2f} seconds")
print(f"Median aller Spark Jobs: {np.median(listcount):.2f} seconds")
print(f"Sdt aller Spark Jobs: {np.std(listcount):.2f} seconds")

  Included Job 137 took 0.05s
  Included Job 136 took 9.55s
  Included Job 135 took 8.82s
  Included Job 134 took 12.49s


  Included Job 140 took 0.05s
  Included Job 139 took 9.49s
  Included Job 138 took 8.40s


  Included Job 143 took 0.10s
  Included Job 142 took 9.48s
  Included Job 141 took 9.27s


  Included Job 146 took 0.08s
  Included Job 145 took 9.05s
  Included Job 144 took 7.98s


  Included Job 149 took 0.03s
  Included Job 148 took 9.24s
  Included Job 147 took 8.47s


  Included Job 152 took 0.07s
  Included Job 151 took 10.01s
  Included Job 150 took 8.42s


  Included Job 155 took 0.07s
  Included Job 154 took 8.95s
  Included Job 153 took 7.96s


  Included Job 158 took 0.07s
  Included Job 157 took 9.71s
  Included Job 156 took 7.68s


  Included Job 161 took 0.04s
  Included Job 160 took 9.29s
  Included Job 159 took 9.24s


  Included Job 164 took 0.04s
  Included Job 163 took 9.29s
  Included Job 162 took 7.91s
Average time over 10 queries: 19.13 seconds
[30.915, 17.945, 18.855, 17.101, 17.748, 18.491, 16.986, 17.461, 18.562, 17.246]
Zeit für die Ausführung der Zelle: 205.00 seconds
Mittelwert aller Spark Jobs: 19.13 seconds
Median aller Spark Jobs: 17.85 seconds
Sdt aller Spark Jobs: 3.98 seconds
